# Bienestar en Bogotá - OPM
## Juan Diego Mayorga 
## MCPP
## 2022-1
-------------------
## Códigos Alternativos
Este código presentará las alternativas que se tuvieron en cuenta para la elaboración del proyecto. 

In [2]:
# Packages 

# OSM
#%pip install overpass
#%pip install OSMPythonTools]

import requests 
import overpass
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

# Data 
import pandas as pd
import json
from datetime import datetime,timedelta
import time

# Text 
import re

# Graphs 
import matplotlib.pyplot as plt
import seaborn as sns

### 0. Conexión con la API

In [2]:
# Nos conectamos con la API
api = overpass.API(endpoint="https://overpass.myserver/interpreter")
api = overpass.API(timeout=600)

# Dejamos el link abierto para los request 
op_url = "https://overpass-api.de/api/interpreter" 

### 1. Descarga de los datos
La API es muy sensible, a continuación un par de errores comunes que encontré 

1. Tildes: toca escribir tal cual se encuentra la ubicación en OSM
2. Palabras extra: por ejemplo, unos querys iniciales que hice tenían escrito "Localidad Candelaria", pero en OSM aparece "Localidad La Candelaria". Importante tener en cuenta estas variaciones.
2. No todas las localidades se escriben o se encuentran de la misma forma.

#### 1.0 Alternativas

Esta sección explicaré las alternativas que pude haber escogido para construir el query 

##### - Buffer 
Siguiendo este post de [Github](https://github.com/nikhilnagargit/OpenStreetMapDataExtraction/blob/master/data_task__by_nikhil.py), la primera aproximación que realicé fue a través de un método de **buffer** en el cual le indicaba a OSM la latitud, longitud y el radio del área que quería analizar y este me arrojaba los resultados. Sin embargo, el principal problema con esta alternativa es que era muy difícil encontrar el centroide de cada una y establecer un radio que no se sobre pusiera a las otras localidades. Por lo tanto, este método quedó descartado después de muchas pruebas. No obstante, recomiendo el código elaborado por @nikhil_nagar porque el trabajo mucho en el output resultado, logra extraer información muy valiosa. 

##### - Datos totales
Como se presenta a continuación, mi segunda aproximación fue hacer un conteo de los datos totales de los amenities que quería estudiar. Esto a través de la sintaxis del código de Overpass, en la cual le pongo al final "out count;". Sin embargo, el problema con esta metodología es que OSM no cuenta aquellos establecimientos que no tienen nombre o suficiente información, por lo tanto, tenía un subregistro del número de elementos que estaba buscando. Adicionalmente, esta metodología no me permitía construir mapas de calor dado que solo me arrojaba 19 datos, uno por cada localidad.

A continuación presento el código de esta metodología. Esta muy avanzada, incluso logré generar outputs a partir de esta metodología.

**Datos totales**

En esta sección explicaré como construiré los querys para descargar los datos TOTALES de los amenitys que me interesan de Bogotá por localidad

In [3]:
# Lista de las localidades para hacer un loop 
# Excluimos Sumapaz 
localidades = ["'Localidad Usaquén'", "'Localidad Chapinero'", "'Localidad Santa Fé'", "'Localidad San Cristóbal'", "'Localidad Usme'", "'Localidad Tunjuelito'",
    "'Localidad Bosa'", "'Localidad Kennedy'", "'Localidad Fontibón'", "'Localidad Engativá'", "'Localidad Suba'", "'Localidad Barrios Unidos'",
    "'Localidad Teusaquillo'", "'Localidad Los Mártires'", "'Localidad Antonio Nariño'", "'Localidad Puente Aranda'", "'Localidad La Candelaria'",
    "'Localidad Rafael Uribe Uribe'", "'Localidad Ciudad Bolivar'"]

# Las localidades estan en orden, necesitamos poner un indicador de la zona para lograr un merge con datos administrativos mas adelante
# Dado que vamos a hacer pruebas y el len puede que cambie a partir de la lista de localidades, vamos a hacer un comando que tome el tamaño de la lista 
index_loc = list(range(1,len(localidades)+1))
index_loc

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [4]:
# Primero hagamos unas pruebas 

# Esta es la estructura básica del lenguaje de programación OSM. Puedo preguntarle por nodos, areas, ways y entre otros. En este caso vamos
# A probar ways

response = api.get('node["name"="Localidad Los Mártires"]')
response
# Obtenemos info sobre la localidad los mártiles en bogotá

{"features": [{"geometry": {"coordinates": [-74.085471, 4.604828], "type": "Point"}, "id": 7641780773, "properties": {"name": "Localidad Los M\u00e1rtires", "place": "suburb", "population": "99119", "wikidata": "Q2648349", "wikipedia": "es:Los M\u00e1rtires"}, "type": "Feature"}], "type": "FeatureCollection"}

In [98]:
# Agreguemos información adicional a nuestro query como lo puede ser el fomato para exportar las tablas (json), el area donde queremos que busque (e.g Bogotá)
# Y el tipo de amenity que queremos encontrar (restaurantes)

test = ([])

op_query ="""
    [out:json][timeout:25];
    area[name='Bogotá']->.search;
    node["amenity"="restaurant"](area.search);
    out count;out skel qt;""" # Count se puede cambiar

op_query

# Aquí utilizamos la función request con su argumento get para realizar el query
response = requests.get(op_url, params={"data": op_query})

# Esta estructura permite transformar los datos de json a data frame
x = response.json()
data = pd.json_normalize(x["elements"])

# Pegamos todos los datos en el vector vacio que construimos arriba
test.append(int(data['tags.total'].values[0]))
test

# Como se puede apreciar, en bogotá hay alrededor de 3212 restaurantes
# OJO, estos son restaurantes registrados. En principio hay 6432 pero OSM solo arroja aquellos que tienen info registrada (nombre, ubicación)

[3216]

El siguiente código se realizó a partir de la presentación del profesor Nikolai Janakiev "Data Science with OpenStreetMap and Python (Maptime Salzburg 2018)"

Puede encontrar esta ppt en el siguiente [link](https://www.youtube.com/watch?v=WmCLQCohL3k&t=96s&ab_channel=MaptimeSalzburg]).


In [285]:
# Ahora hagamos un loop para cada uno de las localidades. Esto es una prueba para verificar que todas las localidades esten funcionando de forma correcta 
# Esto es una replicación del código anterior pero aplicado a un loop, esto para que no tengamos que poner cada localidad para un solo amenity

restaurantes = []

for i in localidades:

    p1 = """[out:json][timeout:50];"""
    p2 = """area[name=""" + i + """]->.search;"""
    p3 =  """node["amenity"="restaurant"](area.search);out count;out skel qt;"""

# Tenemos que construir el query por partes porque el loop depende directamente de un str, entonces para incluir las localidades debemos realizar esta modifiación
    op_query = p1 + p2 + p3
    
# Aquí utilizamos la función request con su argumento get para realizar el query
    response = requests.get(op_url, params={"data": op_query})

# Esta estructura permite transformar los datos de json a data frame
    x = response.json()

# Pegamos todos los datos en el vector vacio que construimos arriba
    data = pd.json_normalize(x["elements"])
    restaurantes.append(int(data['tags.total'].values[0]))

# La API no permite hacer cierto numero de querys al tiempo, por tanto, toca poner a dormir al código para que no identifique nuestras multiples solicitudes    
    time.sleep(8)

# Guardamos todo en un df
# Le agregamos un identificador de la localidad y su nombre
rest = pd.DataFrame(
    {"Restaurantes": restaurantes, "Localidad": index_loc, "Nom_localidad": localidades})

In [56]:
# Ahora generaré una función que permita descargar cualquier tipo de dato en relación a amenity's 

def get_amenity(amenity):

    '''
    La siguiente función le permite al usaurio descargar diferentes amenitys para distintas areas del mundo. 
    El único argumento es el nombre del amenity. 
    Si lo desea, puede ajustar las localidades, esto creando una lista con dichas ubicaciones. 
    Para saber que tipo de información se puede seleccionar por favor revise el siguiente link: https://wiki.openstreetmap.org/wiki/Key:amenity
    '''

    y = []
    for i in localidades:
        
# Dado que debemos meterle una parte en código al str en lenguase OSM debemos hacer la siguiente opreación 
        p1 = """[out:json][timeout:50];"""
        p2 = """area[name=""" + i + """]->.search;"""
        p3 =  """node["amenity"=""" + amenity + """](area.search);out count;"""

        op_query = p1 + p2 + p3
        
        response = requests.get(op_url, params={"data": op_query})
        x = response.json()
        data = pd.json_normalize(x["elements"])
        y.append(int(data['tags.total'].values[0]))

# La API no permite hacer cierto numero de querys al tiempo, por tanto, toca poner a dormir al código para que no nos saque 
# Este puede ir cambiando, dado que vamos a hacer multiples request, puede que arroje errores a la hora de calcularlo 
        time.sleep(5)
    data_id = pd.DataFrame(
    {amenity: y, "Localidad": index_loc, "Nom_localidad": localidades})

    return data_id

In [59]:
# Prueba para saber si la función es correcta 
restaurantes = get_amenity("restaurant")

----------
**Creación de las bases de datos** 

Vamos a hacer por grupos de interés las bases de datos y posteriormente se unirán todas 

**Entretenimiento, Arte y Cultura**

In [9]:
# Base de datos de los ameneity que corresponden a entrenimineto, arte y cultura 
casino = get_amenity("casino")
cinema = get_amenity("cinema")
community_centre = get_amenity("community_centre")
love_hotel = get_amenity("love_hotel")
nightclub = get_amenity("nightclub")
theatre = get_amenity("theatre")

In [16]:
# Guardamos la base de datos de entretenimiento
entre = pd.concat([cinema, community_centre, love_hotel, nightclub, theatre], axis=1)

# Eliminamos los duplicados 
entre = entre.loc[:,~entre.columns.duplicated()]

# Creamos una variable que sume el total de amenitys 
entre["Total Entretenimiento"] = entre["cinema"] + entre["community_centre"] + entre["love_hotel"] + entre["nightclub"] + entre["theatre"]

entre.to_csv("Data sets/Entretenimiento.csv", sep = ";", index=False, encoding='utf-8-sig')

**Comida**

In [20]:
bar = get_amenity("bar")
pub = get_amenity("pub")
cafe = get_amenity("cafe")
fast_food = get_amenity("fast_food")
restaurant = get_amenity("restaurant")

In [21]:
# Guardamos la base de datos de comida
comida = pd.concat([bar, pub, cafe, fast_food, restaurant], axis=1)
comida = comida.loc[:,~comida.columns.duplicated()]

# Creamos una variable que sume el total de amenitys 
comida["Total Comida"] = comida["bar"] + comida["pub"] + comida["cafe"] + comida["fast_food"] + comida["restaurant"] 

comida.to_csv("Data sets/Comida.csv", sep = ";", index=False, encoding='utf-8-sig')

**Education**

In [6]:
uni_1 = get_amenity("college")
uni_2 = get_amenity("university")
pre_school = get_amenity("kindergarten")
colegio = get_amenity("school")

In [8]:
# Guardamos la base de datos de educación 
edu = pd.concat([uni_1, uni_2, pre_school, colegio], axis=1)
edu = edu.loc[:,~edu.columns.duplicated()]

# Creamos una variable que sume el total de amenitys 
# Para educación crearemos dos variables, una con superior y otra primaria/secundaria
edu["Primaria"] = edu["kindergarten"] + edu["school"]
edu["Superior"] = edu["college"] + edu["university"]

edu.to_csv("Data sets/Educación.csv", sep = ";", index=False, encoding='utf-8-sig')